<a href="https://colab.research.google.com/github/FleaBusyBeeBergs/dtsa5506-pipeline/blob/main/dtsa5506-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment to install

# !pip install pandas requests pandasdmx
# !pip install requests-cache

In [1]:
import zipfile
import pandas as pd
import os
import requests
# import pandasdmx # sdmx api library
# from pandasdmx import Request
# from requests_cache import install_cache
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

## The Data

The data used comes from the statistics canada data cube (herein referred to as the 'Cube') and is found in the following tables:

* wages, table 14-10-0223-01
* raw materials index, table 18-10-0268-01
* taxes, table 11-10-0058-01
* productivity, table 36-10-0206-01
* consumer price index, table 18-10-004-01

## ETL

There is a very large amount of data in the Cube. Pulling an entire table then paring it down to a few variables is inefficient - create an ETL pipeline.

In [10]:
# create df of tables
tables = {
    'name': ['wage', 'raw', 'tax', 'productivity', 'cpi'],
    'tableid': [14100223, 18100268, 11100058, 36100206, 18100004],
    'vector': ['v79311153', 'v1230998135', 'v122807833', 'v1409153', 'v41690973'],
    'description': ['', '', '', '', ''],
    'frequency': ['monthly', 'monthly', 'annual', 'quarterly', 'monthly'],
    'start': ['2001-01', '2001-01', 'YYYY', 'YYYY-Qx', '2001-01'],
    'end': ['2024-12', '2024-12', 'YYYY', 'YYYY-Qx', '2024-12'],
    'url': ['', '', '', '', '']
         }
table_df = pd.DataFrame(tables)
print(table_df)

# base_url = 'https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/vector/'

           name   tableid       vector description  frequency    start  \
0          wage  14100223    v79311153                monthly  2001-01   
1           raw  18100268  v1230998135                monthly  2001-01   
2           tax  11100058   v122807833                 annual     YYYY   
3  productivity  36100206     v1409153              quarterly  YYYY-Qx   
4           cpi  18100004    v41690973                monthly  2001-01   

       end url  
0  2024-12      
1  2024-12      
2     YYYY      
3  YYYY-Qx      
4  2024-12      


In [20]:
url = 'https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/vector/v122807833?startPeriod=2001-01&endPeriod=2022-01&detail=full'

response = requests.get(url)
response

<Response [200]>

In [12]:
# Parse XML
root = ET.fromstring(response.content)

# Print all elements
for child in root:
    print(child.tag, child.text)

{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}Header None
{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}DataSet None


In [13]:
# Parse the XML response
root = ET.fromstring(response.content)

# Define the namespace
namespace = {'ns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message'}

# Print the contents of the Header
header = root.find('ns:Header', namespace)
if header is not None:
    for elem in header:
        print(f"Header Element: {elem.tag}, Value: {elem.text}")

# Print the contents of the DataSet
dataset = root.find('ns:DataSet', namespace)
if dataset is not None:
    for elem in dataset:
        print(f"DataSet Element: {elem.tag}, Value: {elem.text}")


Header Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}ID, Value: DS8394833387236
Header Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}Test, Value: false
Header Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}Prepared, Value: 2025-01-21T18:56:32
Header Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}Sender, Value: None
Header Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}Structure, Value: None
DataSet Element: {http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic}Series, Value: None


In [14]:
def print_elements(element, level=0):
    indent = "  " * level
    print(f"{indent}Tag: {element.tag}, Text: {element.text}")
    for child in element:
        print_elements(child, level + 1)

# Explore the root element
#print_elements(root)


In [15]:
with open("response_output.xml", "wb") as file:
    file.write(response.content)

In [17]:
# Define the namespace mappings
namespaces = {
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
}

# Parse the XML content
root = ET.fromstring(response.content)  # Replace 'xml_content' with your XML string or response.content

# Extract SeriesKey
series = root.find(".//generic:Series", namespaces)
if series is not None:
    series_key = series.find(".//generic:SeriesKey", namespaces)
    print("Series Key:")
    for value in series_key.findall(".//generic:Value", namespaces):
        print(f"  {value.attrib['id']}: {value.attrib['value']}")

# Extract Attributes
attributes = series.find(".//generic:Attributes", namespaces)
print("\nAttributes:")
for attr in attributes.findall(".//generic:Value", namespaces):
    print(f"  {attr.attrib['id']}: {attr.attrib['value']}")

# Extract Observations
print("\nObservations:")
observations = series.findall(".//generic:Obs", namespaces)
for obs in observations:
    obs_dim = obs.find(".//generic:ObsDimension", namespaces)
    obs_val = obs.find(".//generic:ObsValue", namespaces)
    #print(f"  {obs_dim.attrib['value']}: {obs_val.attrib['value']}")


Series Key:
  Geography: 2
  Products_and_product_groups: 2

Attributes:
  VECTOR_ID: 41690973
  SCALAR_FACTOR: 0
  NB_DECIMAL: 1
  DGUID: 2016A000011124
  UOM: 17

Observations:


In [19]:
# Collect observation data
data = []
for obs in observations:
    obs_dim = obs.find(".//generic:ObsDimension", namespaces)
    obs_val = obs.find(".//generic:ObsValue", namespaces)
    data.append({"Date": obs_dim.attrib["value"], "Value": float(obs_val.attrib["value"])})

# Create DataFrame
df = pd.DataFrame(data)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    264 non-null    object 
 1   Value   264 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.3+ KB
None


,Date,Value
0,2001-01,96.3
1,2001-02,96.8
2,2001-03,97.1
3,2001-04,97.8
4,2001-05,98.6


In [ ]:
cpi_df = df.loc[1044:1319,]

cpi_df.head


In [ ]:
cpi_df.to_csv('cpi.csv', index = False)

### Cleaning and wrangling

In [ ]:
wage = pd.read_csv('data/wage.csv')
wage.rename(columns = {'Value': 'wage'}, inplace = True)
tax = pd.read_csv('data/tax.csv')
tax.rename(columns = {'Value': 'tax'}, inplace = True)
rawmat = pd.read_csv('data/rawmat.csv')
rawmat.rename(columns = {'Value': 'rawmat'}, inplace = True)
prod = pd.read_csv('data/prod.csv')
prod.rename(columns = {'Value': 'prod'}, inplace = True)
cpi = pd.read_csv('data/cpi.csv')
cpi.rename(columns = {'Value': 'cpi'}, inplace = True)

In [ ]:
df = (pd.merge(wage, rawmat,
               on = 'Date',
               how = 'outer'))
df = (pd.merge(df, cpi,
               on = 'Date',
               how = 'outer'))
df

In [ ]:
prod['Date'] = pd.to_datetime(prod['Date'].str[:4] + '-' +
                              prod['Date'].str[-2:].replace({'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}))


In [ ]:
prod['Date'] = prod['Date'].dt.strftime('%Y-%m')

In [ ]:
df = (pd.merge(df, prod,
               on='Date',
               how ='outer'))


In [ ]:
df['prod'] = df['prod'].ffill()

In [ ]:
tax['Date'] = pd.to_datetime(tax['Date'])

In [ ]:
tax

In [ ]:
df

## Train-test-split

## EDA

* Create for loop to visualize series, line plot (interactive?)
* Correlation grid
*

## Preprocessor
define steps so they are applied equally to both training and testing sets.